In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import talib as ta2
import numpy as np
from datetime import datetime as dt
from datetime import timedelta as delta
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
# stock = 'wipro'
# start_date = '2000-01-01'
# end_date = (dt.now() + delta(1)).strftime('%Y-%m-%d')

# df = yf.download(f"{stock}.NS", period='1d', start=start_date, end=end_date)

In [3]:
stocks = ['HCLTECH']
start_date = '2000-01-01'
end_date = (dt.now() + delta(1)).strftime('%Y-%m-%d')

df = pd.concat([yf.download(f"{i}.NS", period='1d', start=start_date, end=end_date) for i in stocks])

[*********************100%%**********************]  1 of 1 completed


In [4]:
def analyse_candlestick_patterns(df, pattern:str):
    x = eval(f"(ta2.CDL{pattern}(df['Open'], df['High'], df['Low'], df['Close'])/100).reset_index()")
    x = x[x[0]==1]
    x.rename(columns = {0:pattern}, inplace = True)
    return x.set_index('Date')

In [5]:
patterns = ['RICKSHAWMAN', 'ABANDONEDBABY', 'HARAMICROSS', 'HARAMI', '3WHITESOLDIERS', 'MORNINGSTAR', 'HAMMER']

for pattern in patterns:
    print(f'running {pattern}')
    _pattern_recognised = analyse_candlestick_patterns(df, pattern)
    df = df.merge(_pattern_recognised, how='left', left_index=True, right_index=True).fillna({pattern:0})
    df = df.astype({pattern:'int'})

running RICKSHAWMAN
running ABANDONEDBABY
running HARAMICROSS
running HARAMI
running 3WHITESOLDIERS
running MORNINGSTAR
running HAMMER


In [6]:
df

,Open,High,Low,Close,Adj Close,Volume,RICKSHAWMAN,ABANDONEDBABY,HARAMICROSS,HARAMI,3WHITESOLDIERS,MORNINGSTAR,HAMMER
Date,,,,,,,,,,,,,
2002-08-12,22.712500,23.987499,22.625000,23.762501,14.427621,11239400,0,0,0,0,0,0,0
2002-08-13,24.250000,24.250000,23.150000,23.325001,14.161982,3756368,0,0,0,0,0,0,0
2002-08-14,23.237499,23.237499,22.768749,23.006250,13.968454,3964016,0,0,0,0,0,0,0
2002-08-15,23.006250,23.006250,23.006250,23.006250,13.968454,0,0,0,0,0,0,0,0
2002-08-16,23.625000,23.750000,23.250000,23.606251,14.332751,4848904,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-12,1285.000000,1291.750000,1265.500000,1282.400024,1282.400024,2140769,0,0,0,0,0,0,0
2023-09-13,1282.000000,1282.400024,1266.099976,1275.250000,1275.250000,2275523,0,0,0,0,0,0,0
2023-09-14,1280.849976,1290.699951,1275.599976,1283.599976,1283.599976,2655172,0,0,0,0,0,0,0


In [7]:
def check_returns_for_candlestick(df, candlestick_col, holding_period=1):
    df['returns'] = (df['Adj Close'].shift(-holding_period) - df['Adj Close']) / df['Adj Close'] * 100
    return df[df[candlestick_col]==1]

In [8]:
check_returns_for_candlestick(df, 'RICKSHAWMAN', 14)['returns'].mean()

2.0890219379332584

In [9]:
raw_summary = []

for pattern in patterns:
    for days in range(1,15,1):
        _mean = check_returns_for_candlestick(df, pattern, days)['returns'].mean()
        _median = check_returns_for_candlestick(df, pattern, days)['returns'].median()
        raw_summary.append([pattern, days, _mean, _median])

In [10]:
summary = pd.DataFrame(raw_summary, columns=['candlestick pattern', 'days', 'mean', 'median'])\
    .dropna()\
    .sort_values(by='mean', ascending=False)

summary

,candlestick pattern,days,mean,median
97,HAMMER,14,2.160903,2.138606
13,RICKSHAWMAN,14,2.089022,1.665787
96,HAMMER,13,2.045509,1.980100
41,HARAMICROSS,14,2.026239,1.819724
12,RICKSHAWMAN,13,2.015442,1.714977
...,...,...,...,...
77,MORNINGSTAR,8,-0.820240,-0.522040
78,MORNINGSTAR,9,-0.843858,-0.112027
76,MORNINGSTAR,7,-0.939929,-1.182722
74,MORNINGSTAR,5,-0.947526,-0.711710


In [11]:
summary[(summary['mean']>0.5) & (summary['median']>0.5)].sort_values(by=['mean', 'median'], ascending=False)

,candlestick pattern,days,mean,median
97,HAMMER,14,2.160903,2.138606
13,RICKSHAWMAN,14,2.089022,1.665787
96,HAMMER,13,2.045509,1.980100
41,HARAMICROSS,14,2.026239,1.819724
12,RICKSHAWMAN,13,2.015442,1.714977
11,RICKSHAWMAN,12,1.962363,1.559655
10,RICKSHAWMAN,11,1.905431,1.466798
55,HARAMI,14,1.829300,1.662800
38,HARAMICROSS,11,1.820876,0.821325
54,HARAMI,13,1.809651,1.534410


In [12]:
summary.groupby('candlestick pattern').mean().sort_values(by=['mean', 'median'], ascending=False)

,days,mean,median
candlestick pattern,,,
RICKSHAWMAN,7.5,1.198841,1.005025
HARAMI,7.5,1.132660,1.033520
HARAMICROSS,7.5,1.098020,0.805835
HAMMER,7.5,0.846939,1.258641
MORNINGSTAR,7.5,-0.485161,0.293342


In [13]:
summary.groupby('candlestick pattern').median().sort_values(by=['mean', 'median'], ascending=False)

,days,mean,median
candlestick pattern,,,
RICKSHAWMAN,7.5,1.182367,1.005527
HARAMI,7.5,1.168750,1.034293
HARAMICROSS,7.5,1.078949,0.793811
HAMMER,7.5,0.624444,1.308441
MORNINGSTAR,7.5,-0.341420,0.334658


In [14]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()

In [15]:
summary.groupby('candlestick pattern').apply(weighted_average, 'mean', 'days').sort_values(ascending=False)

candlestick pattern
RICKSHAWMAN    1.541642
HARAMICROSS    1.428531
HARAMI         1.383269
HAMMER         1.123104
MORNINGSTAR   -0.482562
dtype: float64

In [16]:
summary.groupby('candlestick pattern').apply(weighted_average, 'median', 'days').sort_values(ascending=False)

candlestick pattern
HAMMER         1.485018
RICKSHAWMAN    1.269702
HARAMI         1.221090
HARAMICROSS    0.961568
MORNINGSTAR    0.645170
dtype: float64